In [1]:
#Weight & Biases
import wandb

#Yaml
from yaml import load, FullLoader

#Embeddings
from dvats.all import *
from tsai.data.preparation import prepare_forecasting_data
from tsai.data.validation import get_forecasting_splits
from fastcore.all import *

#Dimensionality reduction
from tsai.imports import *

#Clustering
import hdbscan
import fastai
from fastai.tabular import *
import fastcore
import fastshap
import fastinference
import wwf

In [2]:
check_memory_usage = True
if check_memory_usage:
    import nbs_pipeline.utils.memory as mem
    import torch 
    gpu_device = torch.cuda.current_device()
    mem.gpu_memory_status(gpu_device)
#Get W&B API
api = wandb.Api()

## Buscando una combinación buena
cpu_flag = False

if cpu_flag:
    n_neighbors = 15
    min_dist = 0.1
else: 
    n_neighbors = 15 #200
    min_dist = 0.1 #0.0001
config = AttrDict(
    use_wandb = False, # Whether to use or not wandb for experiment tracking
    wandb_group = None, # Whether to group this run in a wandb group
    wandb_entity = os.environ['WANDB_ENTITY'], # The entity to use for wandb,
    wandb_project = os.environ['WANDB_PROJECT'], # The project to use for wandb,
    dr_artifact_name = None, # * Set to None for using the default one (encoder validation set)
    enc_artifact = 'mi-santamaria/deepvats/mvp-SWV:latest', # Name:version of the encoder artifact
    n_neighbors = n_neighbors, #15, #UMAP
    min_dist = min_dist, #0.1, #UMAP,
    random_state = int(1234), # UMAP
    metric = 'euclidean',
    cpu = cpu_flag
)
import os
path = os.path.expanduser("~/work/nbs_pipeline/")
name="to_delete"
runname = name
os.environ["WANDB_NOTEBOOK_NAME"] = path+name+".ipynb"

run_dr = wandb.init(
    entity=config.wandb_entity,
    project=config.wandb_project if config.use_wandb else 'work-nbs', 
    group=config.wandb_group,
    allow_val_change=True, 
    job_type='dimensionality_reduction', 
    mode='online' if config.use_wandb else 'disabled',
    anonymous = 'never' if config.use_wandb else 'must',
    config=config,
    resume = 'allow',
    name = runname
    #resume=False
)
config_dr = wandb.config # Object for storing hyperparameters
artifacts_gettr = run.use_artifact if config_dr.use_wandb else api.artifact
enc_artifact = artifacts_gettr(config.enc_artifact, type='learner')
# TODO: This only works when you run it two timeS! WTF?
try:
    enc_learner = enc_artifact.to_obj()
except:
    enc_learner = enc_artifact.to_obj()

enc_logger = enc_artifact.logged_by()
enc_artifact_train = artifacts_gettr(enc_logger.config['train_artifact'], type='dataset')
if enc_logger.config['valid_artifact'] is not None:
    enc_artifact_valid = artifacts_gettr(enc_logger.config['valid_artifact'], type='dataset')
    print("enc_artifact_valid:", enc_artifact_valid.name)
    print("enc_artifact_train: ", enc_artifact_train.name)


if config_dr.dr_artifact_name is not None:
    dr_artifact = artifacts_gettr(config_dr.dr_artifact_name)
else:
    dr_artifact = enc_artifact_train
df = dr_artifact.to_df()

GPU | Used mem: 5937
GPU | Used mem: 24576
GPU | Memory Usage: [████----------------] 24%


wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  


In [3]:
df.head()

,0,1,2,3,4
2002-01-01 00:00:00,4995.400909,3476.007487,3156.216405,1093.949562,302.794642
2002-01-01 00:00:20,3923.444920,2885.023318,2777.034856,789.073860,277.451848
2002-01-01 00:00:40,4129.310217,2733.710683,2884.988168,736.149227,301.431725
2002-01-01 00:01:00,5329.409519,2877.257957,3593.461371,780.097337,377.047302
2002-01-01 00:01:20,6150.810094,3060.637287,4012.004044,849.544364,406.177858


In [4]:
df.shape

(46455, 5)

In [5]:
df.iloc[10]

0    4222.727052
1    3271.402107
2    3053.700085
3     904.182896
4     257.158857
Name: 2002-01-01 00:03:20, dtype: float64

In [20]:
enc_learner

In [6]:
enc_input, _ = prepare_forecasting_data(df, fcst_history = enc_artifact.metadata['w'])

In [7]:
stride = enc_artifact.metadata['stride']
batch_size = enc_artifact.metadata['batch_size']
print(stride)
print(batch_size)

450
512


In [22]:
X_train = enc_input[::1]

In [23]:
X_train.shape

(45555, 5, 900)

In [41]:
type(X_train)

numpy.ndarray

In [24]:
import numpy as np
import shap
from fastai.tabular.all import *
import fastshap

In [44]:
# SHAP format
#X_train.columns = ["feature_{}".format(i) for i in range(X_train.shape[1])]

# Define una función para obtener predicciones de tu modelo
def model_predict(X):
    return enc_learner.predict(X)

# Create FastShap object
#explainer = shap.Explainer(model_predict, X_train)
explainer = shap.Explainer(model_predict, X_train)

In [26]:
explainer

In [37]:
sample_indices = range(10)

In [40]:
shap_values = explainer.shap_values(X_train[sample_indices,:,:])

TypeError: 'numpy.ndarray' object is not callable

In [27]:
X_train.shape

(45555, 5, 900)

In [79]:
! pip install torchinfo

In [14]:
# Reemplaza 'enc_learner' con tu modelo InceptionTimePlus
enc_learner.summary()

ValueError: This DataLoader does not contain any batches

In [28]:
X_test = X_train[:,:,0]
X_test.shape

(45555, 5)

In [29]:
X_test = pd.DataFrame(X_test)

In [34]:
explainer

In [32]:
shap_values = explainer(X_train)

TypeError: 'numpy.ndarray' object is not callable